In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


# **Загрузка библиотек**

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import string
import re
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_distances
from sklearn.decomposition import PCA
from tqdm import tqdm

In [ ]:
data = pd.read_json('/content/gdrive/MyDrive/Hacki/TalentCaseContest2023/sample.json')

In [ ]:
data.head()

,id,text
0,1,Ты нашёл их или нет?
1,2,Почему она так со мной поступает?
2,3,Никто туда больше не ходит.
3,4,У него с собой не было тогда денег.
4,5,Почему они с нами так поступают?


# **Класс решения**

In [ ]:
class Checker():
    def __init__(self, data):
        '''
        data - pd.DataFrame со столбцами (id, text)
        '''
        # массив словарей {'id': id_text, 'text': text}
        self.id_text = [{'id': data.iloc[i]['id'],
                      'text': "".join(c for c in data.iloc[i]['text'] if (c.isalpha() or c == ' ')).lower()} for i in range(data.shape[0])]

        # словарь {id_text: text}
        self.id_to_text = {data.iloc[i]['id'] : data.iloc[i]['text'] for i in range(data.shape[0])}

        # pd.DataFrame ответа
        self.answer = data

        # все тексты
        self.all_text = None

        # векторы слов
        self.vector_words = None

    def levenshtein_distance(self, sent1: str, sent2: str) -> int:
        '''
        Метод для нахождения расстояния Левенштейна, используя три разных способа

        input: два предложения (только буквы)
        return: расстояние Левенштейна в процентах
        '''
        big = max(len(sent1), len(sent2))
        return (big - min(self.levenshtein_distance_back(' '.join(sorted(sent1.split())), ' '.join(sorted(sent2.split()))),
                   self.levenshtein_distance_back(sent1, sent2),
                   self.update_levenshtein_distance_back(' '.join(sorted(sent1.split())), ' '.join(sorted(sent2.split()))))) / big * 100

    def update_levenshtein_distance_back(self, sent1: str, sent2: str) -> int:
        '''
        Метод для нахождения расстояния Левенштейна, который ищет каждому слову из первого
        предложения максимально похожее слово из второго предложения

        input: два предложения (только буквы)
        return: количество различных символов
        '''

        sent1, sent2 = sorted([sent1, sent2], key=lambda x: len(x.split()), reverse=True)
        same = dict()
        all_delta = 0
        used_word2 = [0 for i in range(len(sent2.split()))]
        for word1 in sent1.split():
            delta = np.inf
            same_word = ''
            n = -1
            for i, word2 in enumerate(sent2.split()):
                now = self.levenshtein_distance_back(word1, word2)
                if now < delta and not used_word2[i]:
                    delta = now
                    same_word = word2
                    n = i
            if n != -1:
                used_word2[n] = 1
            same[word1] = same_word
            if delta != np.inf:
                all_delta += delta
            else:
                all_delta += len(word1)
        return all_delta


    def levenshtein_distance_back(self, sent1, sent2):
        '''
        Метод для нахождения расстояния Левенштейна

        input: два предложения (только буквы)
        return: количество различных символов
        '''

        len1, len2 = len(sent1), len(sent2)
        dp = [[0 for _ in range(len2 + 1)] for _ in range(len1 + 1)]
        for i in range(len1 + 1):
            for j in range(len2 + 1):
                if i == 0:
                    dp[i][j] = j
                elif j == 0:
                    dp[i][j] = i
                elif sent1[i - 1] == sent2[j - 1]:
                    dp[i][j] = dp[i - 1][j - 1]
                else:
                    dp[i][j] = 1 + min(dp[i][j - 1], dp[i - 1][j], dp[i - 1][j - 1])

        return dp[len1][len2]

    def levenshtein_checker(self, threshold=80) -> None:
        '''
        Метод для поиска в data близких предложений по рассстоянию Левенштейна
        threshold - минимальный порог различия в процентах
        '''

        same_dict = dict()
        text_1, text_2 = self.id_text.copy(), self.id_text.copy()

        for i in tqdm(range(len(text_1))):
            for j in range(len(text_2)):
                value = self.levenshtein_distance(text_1[i]['text'], text_2[j]['text'])
                if value >= threshold and text_1[i]['id'] != text_2[j]['id']:
                    if same_dict.get(text_1[i]['id']):
                        same_dict[text_1[i]['id']].append((text_2[j]['id'], value))
                    else:
                        same_dict[text_1[i]['id']] = [(text_2[j]['id'], value)]

        for key, value in same_dict.items():
            same_dict[key] = sorted(value, key=lambda x : x[1], reverse=True)

        same_advanced_dict = dict()
        for key, value in same_dict.items():
            same_advanced_dict[key] = [(self.id_to_text[pair[0]], pair[1]) for pair in value]
        if 'same_texts_levenshtein' in self.answer:
            self.answer.drop('same_texts_levenshtein', axis=1, inplace=True)

        same_advanced_data = pd.DataFrame({'id': list(same_advanced_dict.keys()),
                                  'same_texts_levenshtein': list(same_advanced_dict.values())})
        self.answer = self.answer.merge(same_advanced_data, how='left', on='id')

    def load_other_text(self, paths: list) -> None:
        '''
        Метод для загрузки текстов

        input: список путей до файлов
        '''

        text = []
        for path in paths:
            with open(path) as f:
                text += f.readlines()[:100000]

        self.all_text = ' '.join(text)
        self.all_text = re.split(r'[.|!|?|…]', self.all_text)
        self.all_text = ["".join(c for c in sent if (c.isalpha() or c == ' ')).lower() for sent in self.all_text]

    def word_to_vector(self) -> None:
        '''
        Метод для перевода слов в вектор
        '''

        count_word_other = Counter(' '.join(self.all_text).split())
        V_DIM_other = sum([v >= 10 for v in count_word_other.values()])
        count_word = Counter(' '.join(sent['text'] for sent in self.id_text).split())
        V_DIM_sample = sum([v >= 1 for v in count_word.values()])

        wordID = dict()
        count_word = count_word_other.most_common(V_DIM_other) + count_word.most_common(V_DIM_sample)
        words = set([pair[0] for pair in count_word])
        V_DIM = len(words)
        for i, word in enumerate(words):
            wordID[word] = {"id": i}

        def word_to_id(word):
            if word in wordID:
                return wordID[word]['id']

        p12 = np.zeros((V_DIM, V_DIM))
        p1 = np.zeros((V_DIM,))

        for sent in self.all_text:
            words = set(sent.split())
            for w1 in words:
                p1[word_to_id(w1)] += 1
                for w2 in words:
                    if w2 in wordID:
                        p12[word_to_id(w1), word_to_id(w2)] += 1

        p12 /= p12.sum()
        p1 /= p1.sum()
        vecs = p12 / (p1.reshape((V_DIM,1)) @ p1.reshape((1, V_DIM)))
        vecs[vecs <= 0] = 1
        vecs = np.log(vecs)
        vecs[vecs <= 0] = 0

        pca = PCA()
        res = pca.fit_transform(vecs)
        E_DIM = 2000
        res = res[:, :E_DIM]
        res = (res-res.mean(axis=0))/(res.std(axis=0) * np.sqrt(E_DIM))

        self.vector_words = pd.DataFrame(res)
        self.vector_words.index = [word for word in list(wordID.keys())]

    def embedding_checker(self, threshold=0.2) -> None:
        '''
        Метод для поиска в data близких предложений по косиносному расстоянию между ними
        с помощью перевода предложений в векторы
        threshold - минимальный порог различия
        '''

        self.word_to_vector()
        def word_to_vec(word):
            if word in self.vector_words.index:
                return self.vector_words.loc[word].values
            print(word)

        vector_sent = []
        for sent in self.id_text:
            words = sent['text'].split()
            vector = sum(word_to_vec(word) for word in words) / len(words)
            vector_sent.append(vector)

        dist_matrix = cosine_distances(vector_sent)

        nearest_dict = {}
        for i in range(len(self.id_text)):
            distances = list((sent['id'], dist_matrix[i][j]) for j, sent in enumerate(self.id_text))
            distances.sort(key=lambda x: x[1])
            nearest_dict[self.id_text[i]['id']] = [(id, dist) for id, dist in distances \
                                                if dist <= threshold and self.id_text[i]['id'] != id]
        nearest_advanced_dict = dict()
        for key, value in nearest_dict.items():
            nearest_advanced_dict[key] = [(self.id_to_text[pair[0]], pair[1]) for pair in value]
            if not len(nearest_advanced_dict[key]):
                nearest_advanced_dict[key] = np.nan

        if 'same_texts_embedding' in self.answer:
            self.answer.drop('same_texts_embedding', axis=1, inplace=True)
        same_advanced_data = pd.DataFrame({'id': list(nearest_advanced_dict.keys()),
                                  'same_texts_embedding': list(nearest_advanced_dict.values())})
        self.answer = self.answer.merge(same_advanced_data, how='left', on='id')
        self.answer['same_texts_embedding'] = self.answer['same_texts_embedding'].map(lambda row: [(sent[0], np.round(sent[1], 3)) for sent in row] if type(row) == list else row)


    def unite_columns(self, row):
        levenshtein = [pair[0] for pair in row['same_texts_levenshtein']] if type(row['same_texts_levenshtein']) == list else row
        embedding = [pair[0] for pair in row['same_texts_embedding']] if type(row['same_texts_embedding']) == list else row
        if type(levenshtein) != list and type(embedding) != list:
            return np.nan
        elif type(embedding) != list:
            return levenshtein
        elif type(levenshtein) != list:
            return embedding
        levenshtein = set(levenshtein)
        embedding = set(embedding)
        common = levenshtein & embedding
        only_in_levenshtein = levenshtein - common
        only_in_embedding = embedding - common
        return list(common) + list(only_in_embedding) + list(only_in_levenshtein)

    def make_result(self):
        if 'result' in self.answer:
            self.answer.drop('result', axis=1, inplace=True)
        self.answer['result'] = self.answer.apply(lambda row: self.unite_columns(row), axis=1)
        return self.answer[['id', 'text', 'result']]

In [ ]:
chk = Checker(data)

In [ ]:
chk.load_other_text(['/content/gdrive/MyDrive/Hacki/TalentCaseContest2023/Твен_Марк_Приключения_Тома_Сойера.txt',
                     '/content/gdrive/MyDrive/Hacki/TalentCaseContest2023/18850745.txt',
                     '/content/gdrive/MyDrive/Hacki/TalentCaseContest2023/80889690.txt'])

In [ ]:
chk.levenshtein_checker(75)

100%|██████████| 412/412 [05:38<00:00,  1.22it/s]


In [ ]:
chk.embedding_checker(0.3)

# **Результат работы**

In [ ]:
chk.make_result()

,id,text,result
0,1,Ты нашёл их или нет?,NaN
1,2,Почему она так со мной поступает?,"[Почему он так со мной поступает?, Почему они ..."
2,3,Никто туда больше не ходит.,[Никто больше туда не ходит.]
3,4,У него с собой не было тогда денег.,[У него тогда не было с собой денег.]
4,5,Почему они с нами так поступают?,"[Почему они так с ним поступают?, Почему она т..."
...,...,...,...
407,408,Рому было нечего сказать.,[Тому было нечего сказать.]
408,409,Том ещё нас не видел.,NaN
409,410,Мы должны всем им помочь.,NaN
410,411,Чего ты свой нос повсюду суёшь?,NaN


In [ ]:
chk.answer

,id,text,same_texts_levenshtein,same_texts_embedding,result
0,1,Ты нашёл их или нет?,NaN,NaN,NaN
1,2,Почему она так со мной поступает?,"[(Почему он так со мной поступает?, 96.875), (...","[(Почему он так со мной поступает?, 0.167)]","[Почему он так со мной поступает?, Почему они ..."
2,3,Никто туда больше не ходит.,"[(Никто больше туда не ходит., 100.0)]","[(Никто больше туда не ходит., 0.0)]",[Никто больше туда не ходит.]
3,4,У него с собой не было тогда денег.,"[(У него тогда не было с собой денег., 100.0)]","[(У него тогда не было с собой денег., 0.0)]",[У него тогда не было с собой денег.]
4,5,Почему они с нами так поступают?,"[(Почему они так с ним поступают?, 93.54838709...","[(Почему они так с ним поступают?, 0.167)]","[Почему они так с ним поступают?, Почему она т..."
...,...,...,...,...,...
407,408,Рому было нечего сказать.,"[(Тому было нечего сказать., 95.83333333333334)]","[(Тому было нечего сказать., 0.251)]",[Тому было нечего сказать.]
408,409,Том ещё нас не видел.,NaN,NaN,NaN
409,410,Мы должны всем им помочь.,NaN,NaN,NaN
410,411,Чего ты свой нос повсюду суёшь?,NaN,NaN,NaN


In [ ]:
chk.answer.to_excel('finish.xlsx')